In [72]:
import urllib2, time, random, re
from bs4 import BeautifulSoup, SoupStrainer
import pandas as pd
import requests
import pickle
import nltk
from sklearn.metrics.pairwise import cosine_similarity
import re
import numpy as np
from fuzzywuzzy import process, fuzz
import sys
sys.setrecursionlimit(10000000)

In [4]:
#find all the links given website and append to all_url list. From that list find all the links that end with /boardgame/somedigits
soup = BeautifulSoup(urllib2.urlopen(urllib2.Request("https://boardgamegeek.com/browse/boardgame")).read())
all_url = []
for i in range(1,6):
    soup = BeautifulSoup(urllib2.urlopen(urllib2.Request("https://boardgamegeek.com/browse/boardgame/page/" + str(i))).read())
    for a in soup.find_all('a', href = True):
        all_url.append(a['href'])
board_games = []
for url in all_url:
    if re.match('/boardgame/\d+\w+', url):
        board_games.append(url)

In [7]:
#get the board game ID numbers
board_games_nums =[]
for b in board_games:
    d = re.search('\d+', b)
    board_games_nums.append(d.group(0))

In [69]:
#find the descriptions of all the games and add to dictionary
games = list(set(board_games_nums))
dic ={}
while len(games)>0:
    #print len(games)
    s = ','.join(games[:25])
    final_url = 'http://www.boardgamegeek.com/xmlapi/boardgame/' + s #+ '?comments=1'
    #print final_url
    soup = BeautifulSoup(urllib2.urlopen(urllib2.Request(final_url)).read())
    for num in games[:25]:
        name_of_game = soup.find('boardgame', {'objectid': num}).find('name', {'primary' : 'true'}).text
        des = soup.find('boardgame', {'objectid': num}).description.text
        dic[name_of_game] = {}
        dic[name_of_game]['description'] = des
        mech = soup.find('boardgame', {'objectid': num}).find_all('boardgamemechanic', text = True)
        m = [t.text for t in mech]
        dic[name_of_game]['mechanism'] = m
        #list_of_comments =[]
        #for c in soup.find('boardgame', {'objectid': num}).find_all('comment', {'usernames': True}):
        #    list_of_comments.append(c.text)
        #dic[name_of_game]['comments'] = ' '.join(list_of_comments)
    games = games[25:]


In [73]:
#saving the dictionary 
pickle_out = open('dic_of_game_desp', 'wb')
pickle.dump(dic, pickle_out)
pickle_out.close()

In [85]:
#loading the dictionary
pickle_in = open('dic_of_game_desp', 'rb')
dic2 = pickle.load(pickle_in)

In [87]:
for key in dic2:
    mech = dic2[key]['mechanism'] 
    m = [t.text for t in mech]
    dic2[key]['mechanism'] = m

In [35]:
#tokenize words
#do some preprocessing with Lemmatizer and Stemming
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
tokenize ={}
tokenize_lem = {}
tokenize_stem = {}
stop_words = stopwords.words('english')
stop_words.append('br')
stop_words = set(stop_words)
tokenizer = RegexpTokenizer(r'\w+')
lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()
for game in dic.keys():
    des = tokenizer.tokenize(dic[game]['description'])
    lower_des = [d.lower() for d in des]
    filtered_des = [w for w in lower_des if not w in stop_words]
    lem_des = [lemmatizer.lemmatize(w) for w in filtered_des]
    stem_des = [ps.stem(w) for w in filtered_des]
    tokenize_lem[game] = nltk.FreqDist(lem_des)
    tokenize[game]= nltk.FreqDist(filtered_des)
    tokenize_stem[game] = nltk.FreqDist(stem_des)

In [36]:
#change tokenize words to Dataframe
tokenize_df = pd.DataFrame.from_dict(tokenize, orient = 'index').fillna(0)
tokenize_stem_df = pd.DataFrame.from_dict(tokenize_stem, orient = 'index').fillna(0)
tokenize_lem_df = pd.DataFrame.from_dict(tokenize_lem, orient = 'index').fillna(0)

In [37]:
r = re.compile('Caverna'.lower())
games = [w.lower() for w in list(tokenize_df.columns.values)]
g = filter(r.match, games)
g

[u'caverna']

In [38]:
#Term frequency
def term_freq(df):
    df.loc[:,:]= df.loc[:,:].div(df.sum(axis=1), axis = 0)
    return(df)

In [39]:
def IDF(df):
    num_doc_with_word = df.astype(bool).sum(axis=0)
    num_games = len(df.index)
    idf = np.log(num_games / (1 + df.astype(bool).sum(axis=0)))
    return(idf)

In [40]:
def tf_idf(tf_df, idf):
    df = tf_df.multiply(idf, axis =1)
    return(df)

In [41]:
t = term_freq(tokenize_df)
idf_t = IDF(t)
t_dif = tf_idf(t, idf_t.values)

In [45]:
ts = term_freq(tokenize_stem_df)
idf_t_s = IDF(ts)
t_dif_s = tf_idf(ts, idf_t_s.values)

In [57]:
tl = term_freq(tokenize_lem_df)
idf_t_l = IDF(tl)
t_dif_l = tf_idf(tl, idf_t_l.values)

In [104]:
game_mech ={}
for game in dic.keys():
    #mech = tokenizer.tokenize(dic[game]['mechanism'])
    lower_mech = [m.lower() for m in dic2[game]['mechanism']]
    game_mech[game] = nltk.FreqDist(lower_mech)

In [105]:
mech_df = pd.DataFrame.from_dict(game_mech, orient = 'index').fillna(0)
m_df = term_freq(mech_df)
idf_m = IDF(m_df)
m_dif = tf_idf(m_df, idf_m.values)

In [111]:
m_df

,hand management,variable player powers,campaign / battle card driven,deck / pool building,dice rolling,partnerships,card drafting,set collection,simultaneous action selection,point to point movement,...,simulation,pattern building,paper-and-pencil,line drawing,time track,stock holding,betting/wagering,trick-taking,pattern recognition,rock-paper-scissors
1775: Rebellion,0.000000,0.166667,0.166667,0.000000,0.166667,0.166667,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0
1830: Railways & Robber Barons,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.25,0.0,0.0,0.0,0.0
1960: The Making of the President,0.333333,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0
1989: Dawn of Freedom,0.250000,0.000000,0.250000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0
51st State: Master Set,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0
7 Wonders,0.200000,0.200000,0.000000,0.000000,0.000000,0.000000,0.200000,0.200000,0.200000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0
7 Wonders Duel,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0
A Feast for Odin,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.166667,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0
A Few Acres of Snow,0.200000,0.000000,0.000000,0.200000,0.000000,0.000000,0.200000,0.000000,0.000000,0.200000,...,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0
A Game of Thrones (first edition),0.142857,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0


In [52]:
def recommend(game, df = t_dif):
    cos = cosine_similarity(df)
    game_names = [c.lower() for c in df.index.values]
    cos_df = pd.DataFrame(cos, index = game_names, columns = game_names)
    g_extract = process.extractOne(str(game).lower() , game_names)
    if fuzz.partial_ratio(str(game), g_extract[0]) < 75:
        print ('This game is not in the top 100 games in BGG, did you mean ' + g_extract[0])
    else:
        top = cos_df.loc[:,g_extract[0]].sort_values(ascending = False)[1:11].index.values
        print'Top 10 for matches for ', g_extract[0]
        for c, value in enumerate(top, 1):
            print c, value

In [55]:
recommend('caverna', df = t_dif_s)

Top 10 for matches for  caverna: the cave farmers
1 vast: the crystal caverns
2 agricola: all creatures big and small
3 agricola (revised edition)
4 summoner wars: guild dwarves vs cave goblins
5 agricola
6 ora et labora
7 antiquity
8 above and below
9 la granja
10 mage knight board game


In [56]:
recommend('caverna', df = t_dif)

Top 10 for matches for  caverna: the cave farmers
1 vast: the crystal caverns
2 summoner wars: guild dwarves vs cave goblins
3 agricola: all creatures big and small
4 agricola (revised edition)
5 agricola
6 ora et labora
7 above and below
8 summoner wars: alliances master set
9 belfort
10 the colonists


In [113]:
recommend('caverna', df = t_dif_l)

Top 10 for matches for  caverna: the cave farmers
1 vast: the crystal caverns
2 summoner wars: guild dwarves vs cave goblins
3 agricola: all creatures big and small
4 agricola (revised edition)
5 agricola
6 ora et labora
7 above and below
8 summoner wars: alliances master set
9 antiquity
10 belfort


In [112]:
recommend('caverna', df = m_df)

Top 10 for matches for  caverna: the cave farmers
1 agricola: all creatures big and small
2 the pillars of the earth
3 vasco da gama
4 caylus
5 brew crafters
6 russian railroads
7 le havre
8 charterstone
9 spyrium
10 fields of arle
